<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_09_10_GraphRAG_from_Web_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to search the Web and ingest web pages, extract entities, and ask a question comparing GraphRAG and RAG.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.


---

Install Graphlit Python client SDK

In [30]:
!pip install --upgrade graphlit-client

Initialize Pyvis

In [31]:
!pip install --upgrade pyvis

Initialize Graphlit

In [32]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Define Graphlit helper functions

In [42]:
from typing import List, Optional

# Create specification for Anthropic Sonnet 3.5
async def create_anthropic_specification_graphrag():
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name="Anthropic Sonnet 3.5 GraphRAG",
        type=enums.SpecificationTypes.COMPLETION,
        serviceType=enums.ModelServiceTypes.ANTHROPIC,
        searchType=enums.ConversationSearchTypes.HYBRID,
        systemPrompt="You are an experienced sports reporter writing an answer to a fan question. Rely only on the content sources provided, but don't mention 'sources' explicitly.  If you're unable to respond, say 'I cannot answer given the sources provided.'.",
        customInstructions="Be verbose and detailed in your response, and write 8-10 verbose paragraphs of useful information for Fantasy Football readers. Include interesting facts or color commentary about any named entities, such as Medical Conditions, Places, Persons and Organizations. For example, if an injury is mentioned, give some color for the uneducated reader to understand the details better. Or, if a football stadium is mentioned, give some color about the city it exists in.",
        anthropic=input_types.AnthropicModelPropertiesInput(
            model=enums.AnthropicModels.CLAUDE_3_5_SONNET,
        ),
        graphStrategy=input_types.GraphStrategyInput(
            type=enums.GraphStrategyTypes.EXTRACT_ENTITIES_GRAPH
        ),
        # NOTE: we expand the vector search chunks to the full sections where the chunk was found to provide greater context for LLM completion
        retrievalStrategy=input_types.RetrievalStrategyInput(
            type=enums.RetrievalStrategyTypes.SECTION
        ),
        # NOTE: optionally, we re-ask the LLM to revise it's response and make it better; makes prompt_conversation slower, but provides better results
#        revisionStrategy=input_types.RevisionStrategyInput(
#            type=enums.RevisionStrategyTypes.REVISE,
#            count=1
#        ),
        # NOTE: we use Cohere to rerank the retrieved sources
        rerankingStrategy=input_types.RerankingStrategyInput(
            serviceType=enums.RerankingModelServiceTypes.COHERE
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_anthropic_specification_rag():
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name="Anthropic Sonnet 3.5 RAG",
        type=enums.SpecificationTypes.COMPLETION,
        serviceType=enums.ModelServiceTypes.ANTHROPIC,
        searchType=enums.ConversationSearchTypes.HYBRID,
        systemPrompt="You are an experienced sports reporter writing an answer to a fan question. Rely only on the content sources provided, but don't mention 'sources' explicitly.  If you're unable to respond, say 'I cannot answer given the sources provided.'.",
        customInstructions="Be verbose and detailed in your response, and write 8-10 verbose paragraphs of useful information for Fantasy Football readers. Include interesting facts or color commentary about any named entities, such as Medical Conditions, Places, Persons and Organizations. For example, if an injury is mentioned, give some color for the uneducated reader to understand the details better. Or, if a football stadium is mentioned, give some color about the city it exists in.",
        anthropic=input_types.AnthropicModelPropertiesInput(
            model=enums.AnthropicModels.CLAUDE_3_5_SONNET,
        ),
        # NOTE: we expand the vector search chunks to the full sections where the chunk was found to provide greater context for LLM completion
        retrievalStrategy=input_types.RetrievalStrategyInput(
            type=enums.RetrievalStrategyTypes.SECTION
        ),
        # NOTE: optionally, we re-ask the LLM to revise it's response and make it better; makes prompt_conversation slower, but provides better results
#        revisionStrategy=input_types.RevisionStrategyInput(
#            type=enums.RevisionStrategyTypes.REVISE,
#            count=1
#        ),
        # NOTE: we use Cohere to rerank the retrieved sources
        rerankingStrategy=input_types.RerankingStrategyInput(
            serviceType=enums.RerankingModelServiceTypes.COHERE
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

# Create specification for Anthropic Sonnet 3.5
async def create_anthropic_specification():
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name="Anthropic Claude Sonnet 3.5",
        type=enums.SpecificationTypes.EXTRACTION,
        serviceType=enums.ModelServiceTypes.ANTHROPIC,
        anthropic=input_types.AnthropicModelPropertiesInput(
            model=enums.AnthropicModels.CLAUDE_3_5_SONNET,
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

# Create entity extraction and entity enrichment workflow using LLM specification
async def create_workflow(specification_id: str):
    if graphlit.client is None:
        return;

    input = input_types.WorkflowInput(
        name="Entity Extraction",
        extraction=input_types.ExtractionWorkflowStageInput(
            jobs=[
                input_types.ExtractionWorkflowJobInput(
                    connector=input_types.EntityExtractionConnectorInput(
                        type=enums.EntityExtractionServiceTypes.MODEL_TEXT,
                        modelText=input_types.ModelTextExtractionPropertiesInput(
                            specification=input_types.EntityReferenceInput(id=specification_id)
                        )
                    )
                )
            ]
        ),
        enrichment=input_types.EnrichmentWorkflowStageInput(
            jobs=[
                input_types.EnrichmentWorkflowJobInput(
                    connector=input_types.EntityEnrichmentConnectorInput(
                        type=enums.EntityEnrichmentServiceTypes.WIKIPEDIA
                    )
                )
            ]
        )
    )

    try:
        response = await graphlit.client.create_workflow(input)

        return response.create_workflow.id if response.create_workflow is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_feed(search_text: str, read_limit: int, workflow_id: str):
    if graphlit.client is None:
        return;

    input = input_types.FeedInput(
        name="Web Search",
        type=enums.FeedTypes.SEARCH,
        search=input_types.SearchFeedPropertiesInput(
            # NOTE: Uncomment one of these to select your search service
            type=enums.SearchServiceTypes.TAVILY,
            #type=enums.SearchServiceTypes.EXA,
            text=search_text,
            readLimit=read_limit
        ),
        workflow=input_types.EntityReferenceInput(
            id=workflow_id
        )
    )

    try:
        response = await graphlit.client.create_feed(input)

        return response.create_feed.id if response.create_feed is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def is_feed_done(feed_id: str):
    if graphlit.client is None:
        return;

    response = await graphlit.client.is_feed_done(feed_id)

    return response.is_feed_done.result if response.is_feed_done is not None else None

async def query_contents_graph(feed_id: str):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.query_contents_graph(
            filter=input_types.ContentFilter(
                feeds=[
                    input_types.EntityReferenceFilter(
                        id=feed_id
                    )
                ]
            ),
            graph=input_types.ContentGraphInput(
                # Uncomment to filter just on software and organization nodes
#                types=[enums.ObservableTypes.SOFTWARE,enums.ObservableTypes.ORGANIZATION]
            )
        )

        return response.contents.graph if response.contents is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def create_conversation(specification_id: str):
    if graphlit.client is None:
        return;

    input = input_types.ConversationInput(
        name="Conversation",
        specification=input_types.EntityReferenceInput(
            id=specification_id
        )
    )

    try:
        response = await graphlit.client.create_conversation(input)

        return response.create_conversation.id if response.create_conversation is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def prompt_conversation(conversation_id: str, prompt: str):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.prompt_conversation(prompt, conversation_id)

        return response.prompt_conversation.message.message if response.prompt_conversation is not None and response.prompt_conversation.message is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_all_workflows():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_workflows(is_synchronous=True)

async def delete_all_contents():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_contents(is_synchronous=True)

async def delete_all_feeds():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_feeds(is_synchronous=True)


Define Graphlit knowledge graph helper functions. Required for rendering the knowledge graph with Pyvis.

In [34]:
import random
import os
import json
from pyvis.network import Network
from typing import Optional

def select_emoji(entity_type):
    # Emoji mappings for observable types
    observable_emoji_map = {
        "CONTENT": "📄",  # Page facing up emoji for generic content
        "LABEL": "🏷️",   # Label emoji for categories or tags
        "PERSON": "🧑",  # Person emoji for individuals
        "ORGANIZATION": "🏢",  # Office building emoji for organizations
        "PLACE": "🌍",  # Globe showing Europe-Africa for places
        "PRODUCT": "🛍️",  # Shopping bags emoji for products
        "SOFTWARE": "💻",  # Laptop emoji for software
        "REPO": "🗂️",  # Card index dividers emoji for repositories
        "EVENT": "🎉",  # Party popper emoji for events
        "MEDICAL_STUDY": "📊",  # Bar chart emoji for medical studies
        "MEDICAL_CONDITION": "🤒",  # Face with thermometer emoji for medical conditions
        "MEDICAL_GUIDELINE": "📜",  # Scroll emoji for medical guidelines
        "MEDICAL_DRUG": "💊",  # Pill emoji for medical drugs
        "MEDICAL_DRUG_CLASS": "🧬",  # DNA emoji for medical drug classes
        "MEDICAL_INDICATION": "🔍",  # Magnifying glass emoji for medical indications
        "MEDICAL_CONTRAINDICATION": "🚫",  # Prohibited emoji for contraindications
        "MEDICAL_TEST": "🧪",  # Test tube emoji for medical tests
        "MEDICAL_DEVICE": "🦾",  # Mechanical arm emoji for medical devices
        "MEDICAL_THERAPY": "🩺",  # Stethoscope emoji for medical therapies
        "MEDICAL_PROCEDURE": "🔧",  # Wrench emoji for medical procedures
    }

    # Return the emoji corresponding to the entity type
    return observable_emoji_map.get(entity_type, "📄")  # Default to page facing up emoji if entity type is unknown

def lookup_node_shape(entity_type, content_type, file_type):
    entity_icon_map = {
        "CONTENT": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf15c", "color": "#aec7e8"}},  # file-text
        "LABEL": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf02b", "color": "#ffbb78"}},   # tag (luggage tag-like)
        "PERSON": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf007", "color": "#98df8a"}},  # user
        "ORGANIZATION": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf1ad", "color": "#ff9896"}},  # building
        "PLACE": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf3c5", "color": "#c5b0d5"}},  # globe-americas
        "PRODUCT": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf1b2", "color": "#c49c94"}},  # cube
        "SOFTWARE": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf085", "color": "#f7b6d2"}},  # cog
        "REPO": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf1c0", "color": "#c7c7c7"}},  # database
        "EVENT": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf073", "color": "#dbdb8d"}},  # calendar
        "MEDICAL_STUDY": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf201", "color": "#17becf"}},  # bar-chart
        "MEDICAL_CONDITION": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf0f1", "color": "#d62728"}},  # heartbeat
        "MEDICAL_GUIDELINE": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf02d", "color": "#9467bd"}},  # book
        "MEDICAL_DRUG": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf484", "color": "#bcbd22"}},  # capsules
        "MEDICAL_DRUG_CLASS": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf471", "color": "#1f77b4"}},  # dna
        "MEDICAL_INDICATION": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf0ae", "color": "#2ca02c"}},  # search
        "MEDICAL_CONTRAINDICATION": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf05e", "color": "#ff7f0e"}},  # ban
        "MEDICAL_TEST": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf469", "color": "#e377c2"}},  # vial
        "MEDICAL_DEVICE": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf6a8", "color": "#7f7f7f"}},  # robot
        "MEDICAL_THERAPY": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf0f9", "color": "#8c564b"}},  # stethoscope
        "MEDICAL_PROCEDURE": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf0ad", "color": "#17becf"}},  # wrench
    }

    content_icon_map = {
        "FILE": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf15c", "color": "#aec7e8"}},  # file
        "PAGE": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf0ac", "color": "#aec7e8"}},  # globe
        "MESSAGE": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf075", "color": "#aec7e8"}},  # comment
        "TEXT": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf15c", "color": "#aec7e8"}},  # file-text
        "POST": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf1ea", "color": "#aec7e8"}},  # newspaper
        "EMAIL": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf0e0", "color": "#aec7e8"}},  # envelope
        "EVENT": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf073", "color": "#aec7e8"}},  # calendar
        "ISSUE": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf188", "color": "#aec7e8"}},  # bug
    }

    file_icon_map = {
        "VIDEO": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf03d", "color": "#aec7e8"}},  # video-camera
        "AUDIO": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf028", "color": "#aec7e8"}},  # volume-up
        "IMAGE": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf03e", "color": "#aec7e8"}},  # picture-o (image)
        "DOCUMENT": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf15b", "color": "#aec7e8"}},  # file-text-o
        "EMAIL": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf0e0", "color": "#aec7e8"}},  # envelope
        "CODE": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf121", "color": "#aec7e8"}},  # code
        "DATA": {"shape": "icon", "icon": {"face": "FontAwesome", "code": "\uf1c0", "color": "#aec7e8"}},  # database
    }

    if file_type is not None:
        return file_icon_map.get(file_type, content_icon_map.get("FILE"))
    elif content_type is not None:
        return content_icon_map.get(content_type, content_icon_map.get("FILE"))
    else:
        return entity_icon_map.get(entity_type, {"shape": "dot"})  # Default to a simple dot shape if the entity type is unknown

def lookup_node_color(entity_type):
    entity_color_map = {
        "CONTENT": "#aec7e8",  # Soft blue
        "LABEL": "#ffbb78",   # Soft orange
        "PERSON": "#98df8a",  # Pale green
        "ORGANIZATION": "#ff9896",  # Soft red
        "PLACE": "#c5b0d5",  # Soft purple
        "PRODUCT": "#c49c94",  # Soft brown
        "SOFTWARE": "#f7b6d2",  # Light pink
        "REPO": "#c7c7c7",  # Light gray
        "EVENT": "#dbdb8d",  # Soft yellow
        "MEDICAL_STUDY": "#17becf",  # Cyan
        "MEDICAL_CONDITION": "#d62728",  # Strong red
        "MEDICAL_GUIDELINE": "#9467bd",  # Medium purple
        "MEDICAL_DRUG": "#bcbd22",  # Olive green
        "MEDICAL_DRUG_CLASS": "#1f77b4",  # Strong blue
        "MEDICAL_INDICATION": "#2ca02c",  # Vivid green
        "MEDICAL_CONTRAINDICATION": "#ff7f0e",  # Vivid orange
        "MEDICAL_TEST": "#e377c2",  # Soft magenta
        "MEDICAL_DEVICE": "#7f7f7f",  # Medium gray
        "MEDICAL_THERAPY": "#8c564b",  # Muted brown
        "MEDICAL_PROCEDURE": "#17becf",  # Cyan (same as MEDICAL_STUDY)
    }

    return entity_color_map.get(entity_type, "#ffffff")  # Default to white if entity type is unknown

def parse_metadata(metadata):
    if metadata is None:
        return None, None

    o = json.loads(metadata)

    return enums.ContentTypes[o["type"]] if "type" in o else None, enums.FileTypes[o["fileType"]] if "fileType" in o else None

def pretty_print_json(dictionary):
    return '\n'.join(f"{key}: {value}" for key, value in dictionary.items() if not key.startswith('@'))

def parse_title(metadata):
    if metadata is None:
        return None

    o = json.loads(metadata)

    if o is not None:
        uri = o["uri"] if "uri" in o else None

        title = pretty_print_json(o)

        if uri is not None:
            return f'URI: {uri}' + '\n' + title
        else:
            return title
    else:
        return None

def parse_label(metadata):
    if metadata is None:
        return None

    o = json.loads(metadata)

    file_name = o["fileName"] if "fileName" in o else None

    label = None

    document = o["document"] if "document" in o else None
    audio = o["audio"] if "audio" in o else None
    video = o["video"] if "video" in o else None

    if document is not None and "title" in document:
        label = document["title"]
    elif video is not None and "title" in video:
        label = video["title"]
    elif audio is not None and "title" in audio:
        label = audio["title"]

    return label if label is not None else file_name

def format_relation(relation: str):
    if relation == "observed-by":
        return None

    return relation.replace("-", " ")

def create_pyvis_contents_graph(graph):
    g = create_pyvis_network()

    if graph.nodes is not None:
        for node in graph.nodes:
            content_type = None
            file_type = None
            label = None
            title = None

            parsed_title = parse_title(node.metadata)

            if parsed_title is not None:
                if node.type == enums.EntityTypes.CONTENT:
                    content_type, file_type = parse_metadata(node.metadata)
                    label = parse_label(node.metadata)
                    title = f'{node.type.name} [{node.id}]\n' + parsed_title
                else:
                    title = f'{node.type.name} [{node.id}]\n' + parsed_title

            shape = lookup_node_shape(node.type.name, content_type, file_type)

            if shape is not None:
                g.add_node(node.id, label=label if label is not None else node.name, shape=shape["shape"], icon=shape.get("icon"), color=lookup_node_color(node.type.name), title=title if title is not None else f'{node.type.name} [{node.id}]')

    if graph.edges is not None:
        for edge in graph.edges:
            # ensure start and end vertex exist in graph
            if not edge.from_ in g.node_ids:
                g.add_node(edge.from_)
            if not edge.to in g.node_ids:
                g.add_node(edge.to)

            relation = format_relation(edge.relation)

            width = 3 if edge.relation != "observed-by" else 1

            g.add_edge(edge.from_, edge.to, label=relation, title=relation, width=width, arrowStrikethrough=False, arrows="middle")

    return g

def create_pyvis_network():
    g = Network(
        notebook=False,
        directed=True,
        cdn_resources="in_line",
        height="900px",
        width="100%",
    )

    return g

Execute Graphlit example

In [35]:
from IPython.display import display, Markdown, HTML
import time

# Remove any existing feeds, contents and workflows; only needed for notebook example
await delete_all_workflows()
await delete_all_contents()
await delete_all_feeds()

print('Deleted all feeds, contents and workflows.')

read_limit = 5 # how many search results to ingest from feed

search_text = "Week 1 results from 2024-2025 NFL season"

extraction_specification_id = await create_anthropic_specification()

if extraction_specification_id is not None:
    print(f'Created extraction specification [{extraction_specification_id}].')

    workflow_id = await create_workflow(extraction_specification_id)

    if workflow_id is not None:
        print(f'Created workflow [{workflow_id}].')

        feed_id = await create_feed(search_text, read_limit, workflow_id)

        if feed_id is not None:
            print(f'Created feed [{feed_id}].')

            # Wait for feed to complete, since ingestion happens asychronously
            done = False
            time.sleep(5)
            while not done:
                done = await is_feed_done(feed_id)

                if not done:
                    time.sleep(2)

            print(f'Completed feed [{feed_id}].')

Deleted all feeds, contents and workflows.
Created extraction specification [166d02bc-9abb-4cad-b23d-1a640376819a].
Created workflow [2b7d125f-c51e-465e-b289-8f53178dd21f].
Created feed [ce98ee80-b323-4b90-9eb6-4032c0de94e7].
Completed feed [ce98ee80-b323-4b90-9eb6-4032c0de94e7].


In [36]:

            # Query the resulting knowledge graph for the feed
            graph = await query_contents_graph(feed_id)

            if graph is not None:
                g = create_pyvis_contents_graph(graph)

                g.set_options("""
                var options = {
                    "physics": {
                        "forceAtlas2Based": {
                            "gravitationalConstant": -50,
                            "centralGravity": 0.01,
                            "springLength": 100,
                            "springConstant": 0.08
                        },
                        "maxVelocity": 50,
                        "solver": "forceAtlas2Based",
                        "timestep": 0.35,
                        "stabilization": {
                            "iterations": 25
                        }
                    }
                    }
                """)

                # render with random file name
                graph_html = g.generate_html(f"graph_{random.randint(0, 1000)}.html")

                # Inject FontAwesome CSS
                font_awesome_link = '<script src="https://kit.fontawesome.com/2c74303849.js" crossorigin="anonymous"></script>'
                graph_html = graph_html.replace('<head>', f'<head>{font_awesome_link}')

                display(HTML(graph_html))

In [43]:
prompt = "What injuries happened on Week 1 of the NFL 2024-2025 season?"

completion_specification_id = await create_anthropic_specification_graphrag()

if completion_specification_id is not None:
    print(f'Created completion specification [{completion_specification_id}].')

    conversation_id = await create_conversation(completion_specification_id)

    if conversation_id is not None:
        print(f'Created conversation [{conversation_id}].')

        message = await prompt_conversation(conversation_id, prompt)

        if message is not None:
            display(Markdown('### GraphRAG Conversation:'))
            display(Markdown(f'**User:**\n{prompt}'))
            display(Markdown(f'**Assistant:**\n{message}'))

completion_specification_id = await create_anthropic_specification_rag()

if completion_specification_id is not None:
    print(f'Created completion specification [{completion_specification_id}].')

    conversation_id = await create_conversation(completion_specification_id)

    if conversation_id is not None:
        print(f'Created conversation [{conversation_id}].')

        message = await prompt_conversation(conversation_id, prompt)

        if message is not None:
            display(Markdown('### RAG Conversation:'))
            display(Markdown(f'**User:**\n{prompt}'))
            display(Markdown(f'**Assistant:**\n{message}'))

Created completion specification [33066821-b09f-49b9-804a-1778e4885f31].
Created conversation [7c4cba99-5c09-4513-ae67-e8e025397d31].


### GraphRAG Conversation:

**User:**
What injuries happened on Week 1 of the NFL 2024-2025 season?

**Assistant:**
The 2024 NFL season kicked off with some unfortunate injury news in Week 1. Most notably, Green Bay Packers quarterback Jordan Love suffered an apparent lower leg injury in the final seconds of their game against the Philadelphia Eagles in Brazil. Further medical evaluation revealed Love sustained a sprained MCL, with an expected recovery timeline of 3-6 weeks.

A sprained MCL, or medial collateral ligament, is a common knee injury for football players. It involves stretching or tearing of the ligament on the inner part of the knee. While not as severe as an ACL tear, it can still sideline players for several weeks depending on the grade of the sprain. Love's injury is particularly impactful as he was set to be the Packers' starting quarterback this season after taking over from Aaron Rodgers.

The injury to Love occurred in a historic setting - the NFL's first-ever game played in Brazil. The international matchup between the Packers and Eagles at São Paulo's Corinthians Arena was part of the league's efforts to expand its global reach. Unfortunately, the excitement of this milestone game was somewhat overshadowed by the injury to Green Bay's young quarterback.

While Love's injury was the most significant reported from Week 1, it's worth noting that the NFL's injury reports are typically more comprehensive after all games have concluded. Teams and medical staff often need time to fully evaluate players and determine the extent of any injuries sustained during gameplay.

The Cincinnati Bengals were likely watching their quarterback Joe Burrow closely in Week 1, given his return from wrist surgery that sidelined him for a significant portion of the 2023 season. While no new injuries were reported for Burrow, his performance and durability will be closely monitored throughout the season.

Injuries are an unfortunate reality in the NFL, and Week 1 often sees a higher rate of injuries as players readjust to the intensity of regular-season play after the preseason. Teams must now navigate these early setbacks while maintaining their competitive edge in the long 18-week season ahead.

The Las Vegas Raiders, now entering their fourth season in their new home city, will be hoping to avoid the injury bug that has plagued them in recent years. As a franchise with a storied history dating back to 1960, the Raiders have seen their fair share of season-altering injuries and will be aiming to keep their key players healthy as they compete in the tough AFC West division.

For teams like the Houston Texans and Carolina Panthers, who are looking to rebuild and improve upon previous disappointing seasons, avoiding major injuries to key players will be crucial. Both franchises are among the youngest in the NFL and are still in the process of establishing themselves as consistent contenders.

Created completion specification [6e602efe-5b89-4a2b-881d-bd3fe039ba67].
Created conversation [093f1045-21bc-4eb5-beb5-da61edb11009].


### RAG Conversation:

**User:**
What injuries happened on Week 1 of the NFL 2024-2025 season?

**Assistant:**
The most significant injury from Week 1 of the 2024 NFL season occurred to Green Bay Packers quarterback Jordan Love. In the final seconds of their game against the Philadelphia Eagles in Brazil, Love suffered an apparent lower leg injury that was later diagnosed as a sprained MCL. This injury is expected to sideline Love for 3-6 weeks, dealing an early blow to the Packers' season hopes.

Love's injury is particularly impactful given his status as the Packers' new starting quarterback following Aaron Rodgers' departure. The sprained MCL, which affects the medial collateral ligament on the inner part of the knee, can significantly impact a quarterback's mobility and ability to plant and throw. Green Bay will now have to navigate the next few weeks with a backup quarterback at the helm.

The timing of Love's injury is especially unfortunate, as it occurred in a historic setting - the NFL's first-ever game in Brazil. The match took place in Sao Paulo, marking a significant milestone in the league's international expansion efforts. The injury put a damper on what was otherwise an exciting, high-scoring affair that showcased the NFL's appeal to a new market.

While Jordan Love's injury was the most notable, it's worth mentioning that the first week of an NFL season often sees a higher rate of injuries as players readjust to the intensity of regular-season play. The long flights to Brazil for both the Packers and Eagles may have also contributed to the physical stress on players, potentially increasing injury risk.

On a positive note, the New York Jets saw the return of quarterback Aaron Rodgers, who had missed the entire 2023 season due to an Achilles tear. Rodgers' comeback is a testament to modern medical advancements and rehabilitation techniques, allowing him to return to the field just a year after what was once considered a potentially career-ending injury.

Another quarterback making a return from injury in Week 1 was Joe Burrow of the Cincinnati Bengals. Burrow had missed a significant portion of the 2023 season due to wrist surgery. His performance in the season opener was closely watched to gauge whether he had fully recovered and could return to his previous form as one of the league's top young quarterbacks.

The NFL's focus on player safety has been a major topic in recent years, with rule changes and improved equipment designed to reduce the risk of injuries. However, the physical nature of football means that injuries remain an unfortunate reality of the sport. Teams must always be prepared with capable backups and adaptable game plans to navigate the challenges posed by player injuries.

As the 2024 season progresses, the impact of these early injuries will become clearer. For the Packers, the next few weeks will be crucial as they attempt to stay competitive in Love's absence. Meanwhile, other teams will be monitoring their own players' health closely, knowing that injuries can dramatically alter the course of a season in the highly competitive NFL landscape.